In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import glob
import os
from itertools import product


In [51]:
width = 6.5  # in inches
height = width * 0.75  # 4:3 aspect ratio
fig_size = (width, height)

reds = sns.color_palette("Reds", 6)
blues = sns.color_palette("Blues", 6)

In [52]:
learning_rates = [0.01, 0.005, 0.001]
weight_decays = [1e-4, 1e-5, 1e-6]
experiments = list(product(learning_rates, weight_decays))

experiments[8]

figure_base_path = "figures/np/"

In [53]:
# Load all loss files from ../losses/np/batchsize-128/
# The files are expected to be in format losses_np_<experiment_nr>_epoch<epoch>.pt
# For the non-private version, we executed 9 experiments, each with 100 epochs.
# For now we will only load the losses at epoch 100.

loss_files = glob.glob('../losses/np/14-07/losses_np_*_epoch300.pt')

# Load the losses from each file and store them in a dictionary, with the experiment number as key
losses = {}
for file in loss_files:
    experiment_nr = int(file.split('_')[2])  # Extract the experiment number from the filename
    losses[experiment_nr] = torch.load(file)

losses[1]['train_loss']


[15.305708271503383,
 11.487322810368582,
 7.942742848933449,
 5.858235810994874,
 4.66472411643812,
 3.941966109525724,
 3.4616651992613856,
 3.129577719477014,
 2.875642179373215,
 2.6843750688576815,
 2.540783216349639,
 2.414016938100429,
 2.3201288578907495,
 2.2303555109800595,
 2.156692270293118,
 2.0978059041481463,
 2.0355718336983006,
 1.9921377758545518,
 1.9456875393120843,
 1.9070940065843895,
 1.8686998974274334,
 1.8324368678159284,
 1.808341828517148,
 1.7793228309847204,
 1.7519605707548194,
 1.7278026051932698,
 1.70543632717586,
 1.6851277958961193,
 1.6675203908470597,
 1.6432524844862977,
 1.6297316877692112,
 1.6132197006371647,
 1.5982895172324072,
 1.5809824688575653,
 1.5690762218147258,
 1.559734381784765,
 1.5430353853762497,
 1.5305902949586971,
 1.5144135674881831,
 1.5067119797501587,
 1.4956126340965437,
 1.4879256549607547,
 1.472509695559674,
 1.4687306648857976,
 1.4583201904019796,
 1.448916523334658,
 1.4398699669800998,
 1.4309331830375847,
 1.42163

In [54]:
def plot_loss(train_loss, validation_loss, experiment_nr):
    plt.figure(figsize=fig_size)
    plt.plot(train_loss, label='Train Loss', color='blue')
    plt.plot(validation_loss, label='Validation Loss', color='orange')
    plt.title(f'Losses for Experiment {experiment_nr}: weight decay={experiments[experiment_nr-1][0]}, learningrate={experiments[experiment_nr-1][1]}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{figure_base_path}losses_experiment_{experiment_nr}.pdf')
    plt.close()


for i in range(1, 10):
    plot_loss(losses[i]['train_loss'], losses[i]['val_loss'], i)


In [55]:
metric_files = glob.glob('../metrics/np/14-07/metrics_np_*_epoch*.pt')

# Load the metrics from each file and store them in a dictionary, with the experiment number as key 
metrics = {}
for file in metric_files:
    experiment_nr = int(file.split('_')[2])  # Extract the experiment number from the filename
    epoch = int(file.split('_')[-1].replace('epoch', '').replace('.pt', ''))  # Extract the epoch number
    if experiment_nr not in metrics:
        metrics[experiment_nr] = {}
    metrics[experiment_nr][epoch] = torch.load(file, weights_only=False)

metrics[1][100]

{'experiment': 1,
 'epoch': 100,
 'mse': 0.9895796775817871,
 'mae': 0.7962299585342407,
 'rmse': 0.9947761947200924}

In [56]:
def plot_metrics(metrics):
    # Plot the metrics for a specific experiment
    # on the x-axis we have the epochs, on the y-axis we have the metric value
    # The metric values are mse, mae, and rmse
    for experiment_nr in metrics.keys():
        epochs = sorted(metrics[experiment_nr].keys())
        mse = [metrics[experiment_nr][epoch]['mse'] for epoch in epochs]
        mae = [metrics[experiment_nr][epoch]['mae'] for epoch in epochs]
        plt.figure(figsize=fig_size)
        plt.plot(epochs, mse, label='MSE', color='blue')
        plt.plot(epochs, mae, label='MAE', color='orange')
        plt.title(f'Metrics for Experiment {experiment_nr}: weight decay={experiments[experiment_nr-1][0]}, learningrate={experiments[experiment_nr-1][1]}')
        plt.xlabel('Epochs')
        plt.ylabel('Metric Value')
        plt.legend()
        plt.savefig(f'{figure_base_path}metrics_experiment_{experiment_nr}.pdf')
        plt.close()

    

plot_metrics(metrics)

In [57]:
def plot_mse(metrics):
    # Plot the MSE for all experiments
    plt.figure(figsize=fig_size)
    for experiment_nr in metrics.keys():
        epochs = sorted(metrics[experiment_nr].keys())
        mse = [metrics[experiment_nr][epoch]['mse'] for epoch in epochs]
        plt.plot(epochs, mse, label=f'Experiment {experiment_nr}')
    plt.title('MSE for All Experiments')
    plt.xlabel('Epochs')
    plt.ylabel('MSE')
    plt.legend()
    plt.savefig(f'{figure_base_path}mse_all_experiments.pdf')
    plt.close()


metrics.keys()

dict_keys([3, 1, 9, 4, 2, 7, 8, 6, 5])

In [58]:
# Experiment with the minimal metric values 
def minimal_metrics(metrics):
    # Find the experiment number with the minimal MSE, MAE, and RMSE at epoch 100
    min_mse = float('inf')
    min_mae = float('inf')
    min_rmse = float('inf')
    min_mse_experiment = None
    min_mae_experiment = None
    min_rmse_experiment = None
    for experiment_nr in metrics.keys():
        mse = metrics[experiment_nr][300]['mse']
        mae = metrics[experiment_nr][300]['mae']
        rmse = metrics[experiment_nr][300]['rmse']
        if mse < min_mse:
            min_mse = mse
            min_mse_experiment = experiment_nr
        if mae < min_mae:
            min_mae = mae
            min_mae_experiment = experiment_nr
        if rmse < min_rmse:
            min_rmse = rmse
            min_rmse_experiment = experiment_nr
    print(f'Minimal MSE: {min_mse} in Experiment {min_mse_experiment}')
    print(f'Minimal MAE: {min_mae} in Experiment {min_mae_experiment}')
    print(f'Minimal RMSE: {min_rmse} in Experiment {min_rmse_experiment}')

minimal_metrics(metrics)

Minimal MSE: 0.8298844695091248 in Experiment 1
Minimal MAE: 0.7222684621810913 in Experiment 1
Minimal RMSE: 0.910979950113681 in Experiment 1


In [59]:
def minimal_loss(losses):
    # Find the experiment number with the minimal train and validation loss at epoch 100
    min_train_loss = float('inf')
    min_val_loss = float('inf')
    min_train_loss_experiment = None
    min_val_loss_experiment = None
    for experiment_nr in losses.keys():
        train_loss = losses[experiment_nr]['train_loss'][-1]  # Last value is at epoch 100
        val_loss = losses[experiment_nr]['val_loss'][-1]  # Last value is at epoch 100
        if train_loss < min_train_loss:
            min_train_loss = train_loss
            min_train_loss_experiment = experiment_nr
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            min_val_loss_experiment = experiment_nr
    print(f'Minimal Train Loss: {min_train_loss} in Experiment {min_train_loss_experiment}')
    print(f'Minimal Validation Loss: {min_val_loss} in Experiment {min_val_loss_experiment}')

minimal_loss(losses)

Minimal Train Loss: 0.8843784033541316 in Experiment 1
Minimal Validation Loss: 0.8281576169557989 in Experiment 1


In [60]:
for i in range(1, 10):
    print(f'Experiment {i} - MSE: {metrics[i][300]["mse"]}, MAE: {metrics[i][300]["mae"]}, RMSE: {metrics[i][300]["rmse"]}')
    print(f'Experiment {i} - Train Loss: {losses[i]["train_loss"][-1]}, Validation Loss: {losses[i]["val_loss"][-1]}')




Experiment 1 - MSE: 0.8298844695091248, MAE: 0.7222684621810913, RMSE: 0.910979950113681
Experiment 1 - Train Loss: 0.8843784033541316, Validation Loss: 0.8281576169557989
Experiment 2 - MSE: 0.974905788898468, MAE: 0.7888841032981873, RMSE: 0.9873731761084398
Experiment 2 - Train Loss: 1.132909424400521, Validation Loss: 0.9733529802933423
Experiment 3 - MSE: 1.0243383646011353, MAE: 0.8078224658966064, RMSE: 1.0120960253855042
Experiment 3 - Train Loss: 1.1839740049365244, Validation Loss: 1.0203383520390161
Experiment 4 - MSE: 0.9437381029129028, MAE: 0.7762559056282043, RMSE: 0.9714618381145513
Experiment 4 - Train Loss: 1.0875967270111482, Validation Loss: 0.9400128256197842
Experiment 5 - MSE: 1.0202438831329346, MAE: 0.8056095838546753, RMSE: 1.0100712267622192
Experiment 5 - Train Loss: 1.2501224155315667, Validation Loss: 1.0199100833413595
Experiment 6 - MSE: 1.0377110242843628, MAE: 0.8136131167411804, RMSE: 1.018681021853437
Experiment 6 - Train Loss: 1.2757621965246015, Va

In [61]:
# Function to extract the epoch with the minimal validation loss for each experiment
def minimal_val_loss_epoch(losses):
    min_val_loss_epochs = {}
    for experiment_nr in losses.keys():
        val_loss = losses[experiment_nr]['val_loss']
        min_epoch = np.argmin(val_loss) + 1  # +1 to convert from 0-indexed to 1-indexed epoch
        # round the epoch to nearest number % 10 == 0
        min_epoch = round(min_epoch / 10) * 10
        min_val_loss_epochs[experiment_nr] = min_epoch
    return min_val_loss_epochs

min_val_loss_epochs = minimal_val_loss_epoch(losses)

# Function to extract the metric values at the epoch with the minimal validation loss
def metrics_at_min_val_loss(metrics, min_val_loss_epochs):
    metrics_at_min_val = {}
    for experiment_nr, epoch in min_val_loss_epochs.items():
        if epoch in metrics[experiment_nr]:
            metrics_at_min_val[experiment_nr] = metrics[experiment_nr][epoch]
        else:
            print(f"Epoch {epoch} not found for Experiment {experiment_nr}")
    return metrics_at_min_val

metrics_min_val_loss = metrics_at_min_val_loss(metrics, min_val_loss_epochs)

print("Metrics at Minimal Validation Loss:")
for experiment_nr, metric in metrics_min_val_loss.items():
    print(f"Experiment {experiment_nr} metrics at epoch {min_val_loss_epochs[experiment_nr]}: MSE: {metric['mse']}, MAE: {metric['mae']}, RMSE: {metric['rmse']}")



Metrics at Minimal Validation Loss:
Experiment 2 metrics at epoch 300: MSE: 0.974905788898468, MAE: 0.7888841032981873, RMSE: 0.9873731761084398
Experiment 7 metrics at epoch 300: MSE: 1.1638725996017456, MAE: 0.8482017517089844, RMSE: 1.0788292726848607
Experiment 4 metrics at epoch 300: MSE: 0.9437381029129028, MAE: 0.7762559056282043, RMSE: 0.9714618381145513
Experiment 8 metrics at epoch 300: MSE: 1.2185795307159424, MAE: 0.8636190295219421, RMSE: 1.1038928982088536
Experiment 1 metrics at epoch 300: MSE: 0.8298844695091248, MAE: 0.7222684621810913, RMSE: 0.910979950113681
Experiment 6 metrics at epoch 300: MSE: 1.0377110242843628, MAE: 0.8136131167411804, RMSE: 1.018681021853437
Experiment 3 metrics at epoch 280: MSE: 1.0382260084152222, MAE: 0.8138803243637085, RMSE: 1.018933760563081
Experiment 9 metrics at epoch 300: MSE: 1.1995395421981812, MAE: 0.8572567105293274, RMSE: 1.0952349255745002
Experiment 5 metrics at epoch 300: MSE: 1.0202438831329346, MAE: 0.8056095838546753, RMS

In [62]:
losses_no_metadata = torch.load('../losses/np/14-07/no-metadata/losses_np_no_metadata_epoch300.pt')
metric_files_no_metadata = glob.glob('../metrics/np/14-07/no-metadata/metrics_np_*_epoch*.pt')

# Load the metrics from each file and store them in a dictionary, with the experiment number as key 
metrics_no_metadata = {}
for file in metric_files_no_metadata:
    epoch = int(file.split('_')[-1].replace('epoch', '').replace('.pt', ''))  # Extract the epoch number
    metrics_no_metadata[epoch] = torch.load(file, weights_only=False)

metrics_no_metadata

{135: {'epoch': 135,
  'mse': 1.221663236618042,
  'mae': 0.8623573184013367,
  'rmse': 1.1052887571209806},
 70: {'epoch': 70,
  'mse': 1.940195083618164,
  'mae': 1.0398714542388916,
  'rmse': 1.3929088568955845},
 150: {'epoch': 150,
  'mse': 1.171414852142334,
  'mae': 0.848261833190918,
  'rmse': 1.0823192006715643},
 290: {'epoch': 290,
  'mse': 1.0130935907363892,
  'mae': 0.79815673828125,
  'rmse': 1.0065255042652368},
 280: {'epoch': 280,
  'mse': 1.0178463459014893,
  'mae': 0.7998907566070557,
  'rmse': 1.0088837127744155},
 140: {'epoch': 140,
  'mse': 1.2030935287475586,
  'mae': 0.8571821451187134,
  'rmse': 1.0968562024019186},
 60: {'epoch': 60,
  'mse': 2.280853509902954,
  'mae': 1.1157478094100952,
  'rmse': 1.510249485980033},
 125: {'epoch': 125,
  'mse': 1.2659857273101807,
  'mae': 0.8744534254074097,
  'rmse': 1.1251603118267994},
 105: {'epoch': 105,
  'mse': 1.3966084718704224,
  'mae': 0.9089094400405884,
  'rmse': 1.1817819053744318},
 160: {'epoch': 160,
 

In [63]:
def plot_metrics_no_metadata(metrics):
    # Plot the metrics for the no metadata experiment
    epochs = sorted(metrics.keys())
    mse = [metrics[epoch]['mse'] for epoch in epochs]
    mae = [metrics[epoch]['mae'] for epoch in epochs]
    plt.figure(figsize=fig_size)
    plt.plot(epochs, mse, label='MSE', color='blue')
    plt.plot(epochs, mae, label='MAE', color='orange')
    plt.title('Metrics for No Metadata Experiment')
    plt.xlabel('Epochs')
    plt.ylabel('Metric Value')
    plt.legend()
    plt.savefig(f'{figure_base_path}metrics_no_metadata.pdf')
    plt.close()


plot_metrics_no_metadata(metrics_no_metadata)


def plot_loss_no_metadata(losses):
    plt.figure(figsize=fig_size)
    plt.plot(losses['train_loss'], label='Train Loss', color='blue')
    plt.plot(losses['val_loss'], label='Validation Loss', color='orange')
    plt.title('Losses for No Metadata Experiment')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{figure_base_path}losses_no_metadata.pdf')
    plt.close()


plot_loss_no_metadata(losses_no_metadata)

In [64]:
# Compare the metrics of the no metadata experiment with the results from the best experiment including metadata
def compare_metrics(metrics, metrics_no_metadata, best_experiment):
    # Get the metrics for the best experiment
    best_metrics = metrics[best_experiment][300]
    mse_best = best_metrics['mse']
    mae_best = best_metrics['mae']
    rmse_best = best_metrics['rmse']
    
    # Get the metrics for the no metadata experiment at epoch 100
    mse_no_metadata = metrics_no_metadata[300]['mse']
    mae_no_metadata = metrics_no_metadata[300]['mae']
    
    # Create a DataFrame for comparison
    comparison_df = pd.DataFrame({
        'Experiment': ['1', 'No Metadata'],
        'RMSE': [rmse_best, metrics_no_metadata[300]['rmse']],
        'MSE': [mse_best, mse_no_metadata],
        'MAE': [mae_best, mae_no_metadata]
    })
    
    print(comparison_df)

compare_metrics(metrics, metrics_no_metadata, 1)

    Experiment      RMSE       MSE       MAE
0            1  0.910980  0.829884  0.722268
1  No Metadata  1.004345  1.008709  0.796534


In [65]:
# Plot the training and validation losses of the no metadata experiment with the best experiment including metadata
def plot_comparison_loss(losses, losses_no_metadata, best_experiment):
    plt.figure(figsize=fig_size)
    
    # Plot the best experiment
    plt.plot(losses[best_experiment]['train_loss'], label='Train Loss (Best Experiment)', color='blue')
    plt.plot(losses[best_experiment]['val_loss'], label='Validation Loss (Best Experiment)', color='orange')
    
    # Plot the no metadata experiment
    plt.plot(losses_no_metadata['train_loss'], label='Train Loss (No Metadata)', color='blue', linestyle='--')
    plt.plot(losses_no_metadata['val_loss'], label='Validation Loss (No Metadata)', color='orange', linestyle='--')
    
    plt.title('Comparison of Losses: Best Experiment vs No Metadata')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{figure_base_path}comparison_losses.pdf')
    plt.close()

plot_comparison_loss(losses, losses_no_metadata, 1)

In [66]:
# Plot the training and validation losses of the no metadata experiment with the best experiment including metadata
def plot_comparison_loss(losses, losses_no_metadata, best_experiment):
    plt.figure(figsize=fig_size)
    
    # Only plot the last 100 epochs
    train_loss_best = losses[best_experiment]['train_loss'][-100:]
    val_loss_best = losses[best_experiment]['val_loss'][-100:]
    train_loss_no_metadata = losses_no_metadata['train_loss'][-100:]
    val_loss_no_metadata = losses_no_metadata['val_loss'][-100:]
    epochs = range(len(losses[best_experiment]['train_loss']) - 99, len(losses[best_experiment]['train_loss']) + 1)
    
    plt.plot(epochs, train_loss_best, label='Train Loss (Best Experiment)', color='blue')
    plt.plot(epochs, val_loss_best, label='Validation Loss (Best Experiment)', color='orange')
    plt.plot(epochs, train_loss_no_metadata, label='Train Loss (No Metadata)', color='blue', linestyle='--')
    plt.plot(epochs, val_loss_no_metadata, label='Validation Loss (No Metadata)', color='orange', linestyle='--')
    
    plt.title('Comparison of Losses (Last 100 Epochs): Best Experiment vs No Metadata')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'{figure_base_path}comparison_losses_last100.pdf')
    plt.close()

plot_comparison_loss(losses, losses_no_metadata, 1)